In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("./storage/Kaggle_M5_Luke/sales_train_validation.csv")
df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data) 
print(time_series_data.shape)

(30490, 1913)


In [5]:
# x_train: d_i to d_(i+6), y_train: d_(i+1) to d_(i+7), if time_steps = 7 
# leave last 7 day window for validation 
def split_xy1(dataset,time_steps): 
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps-1): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+1:i+time_steps+1]) 
    return np.array(x),np.array(y)   

x_train, y_train = split_xy1(time_series_data,7)
x_train = x_train.reshape(1905,7,30490) 
y_train = y_train.reshape(1905,7,30490)


x_val = time_series_data[:,1905:1912]  
y_val = time_series_data[:,1906:1913]   
x_val = x_val.reshape(1,7,time_series_data.shape[0]) # time_series_data.shape[0] = 30490
y_val = y_val.reshape(1,7,time_series_data.shape[0]) 
print(x_train.shape) 
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(1905, 7, 30490)
(1905, 7, 30490)
(1, 7, 30490)
(1, 7, 30490)


In [6]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(time_steps,features),activation='relu',return_sequences=True)) 
    model.add(Dense(features)) 
    return model 
model = build_model(7,30490)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 8)              975968    
_________________________________________________________________
dense_1 (Dense)              (None, 7, 30490)          274410    
Total params: 1,250,378
Trainable params: 1,250,378
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='mse',optimizer='adam',metrics=['mse'])
model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base1/model{epoch:02d}.h5',period=50)
model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  



Train on 1905 samples, validate on 1 samples
Epoch 1/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 14.1481 - mse: 14.1481 - val_loss: 9.7340 - val_mse: 9.7340
Epoch 2/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.7184 - mse: 12.7184 - val_loss: 9.6345 - val_mse: 9.6345
Epoch 3/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.4632 - mse: 12.4632 - val_loss: 9.6450 - val_mse: 9.6450
Epoch 4/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.2996 - mse: 12.2996 - val_loss: 9.5501 - val_mse: 9.5501
Epoch 5/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.2301 - mse: 12.2301 - val_loss: 9.5182 - val_mse: 9.5182
Epoch 6/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.1938 - mse: 12.1938 - val_loss: 9.5367 - val_mse: 9.5367
Epoch 7/1000
1905/1905 [==============================] - 6s 3ms/step - loss: 12.1845 - mse: 12.1845 - val_loss: 9.4966 - val_m